## **Robot Model** 

`RobotModel` is a tiny wrapper around the modeling backend (`casadi_kin_dyn`) that allow you to create symbolical CasAdi model from **urdf** file, then calculate a notions such as **forward and inverse dynamics**, com positions, state space representation etc, one may also calculate **body** specific properties and add **contacts**.

The typical workflow as follows:
* Build the urdf of your robot and create `RobotModel` instance
* Add additional bodies and possibly contacts
* Calculate the all necessary functions with `~.update_model()` method 
* Access to casadi functions that are stored within RobotModel and use them either numerically or symbolically in other CasAdi empowered projects.  

There are also banch of modules that facilitates work with given type of robots, i.e. manipulators, quadrupeds and bipeds.


As example let us create the model of z1 manipulator:

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

### *Note*:
You have to restart kernel after `robot_descriptions` is installed

In [1]:
%%capture
!pip3 install robot_descriptions

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend
from darli.modeling import Functional, Robot
from robot_descriptions import z1_description

model = Functional(CasadiBackend(z1_description.URDF_PATH))
# model = Robot(CasadiBackend(z1_description.URDF_PATH))

One can retrive basic model info as follows:

In [3]:
nq = model.nq  # dimensionality of configuration
nv = model.nv  # dimensionality of generilized velocities
nu = model.nu  # dimensionality  of control inputs
q_min, q_max = model.q_min, model.q_max  # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [4]:
joint_names = model.joint_names  # names of the joints
joint_names

['universe', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6']

In [5]:
model.joint_id(joint_names[nv - 1])  # will return id of the joint by name

5

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [6]:
inertia = model.inertia
gravity_vector = model.gravity
bias_force = model.bias_force
# coriolis_matrix = model.coriolis_matrix
coriolis = model.coriolis

Each of the above define the CasAdi functions:

In [7]:
model.inertia

Function(inertia:(q[6])->(inertia[6x6]) SXFunction)

And can be evaluated both numerically and Functionalally:

In [8]:
import numpy as np
import casadi as cs

# Functionalal computation
print("Functionalal:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
print("Numerical:", inertia(np.random.randn(nq)))

Functionalal: @1=7e-07, @2=sin(q_5), @3=1.6e-07, @4=cos(q_5), @5=((@1*@2)+(@3*@4)), @6=(@5*@2), @7=0.0001468, @8=0.165803, @9=0.00177591, @10=-0.00017355, @11=-0.00143876, @12=((@10*@4)-(@11*@2)), @13=((0.0005386+(@6+@7))+(@8*(@9+sq(@12)))), @14=0.307927, @15=0.07, @16=0.0491595, @17=cos(q_4), @18=sin(q_4), @19=0.425807, @20=(@19*@12), @21=(@15+((@16*@17)-(@18*@20))), @22=(0.0436668-@21), @23=((@16*@18)+(@17*@20)), @24=(0.00364738-@23), @25=((0.00097912+@13)+(@14*(sq(@22)+sq(@24)))), @26=cos(q_3), @27=2, @28=1.22e-06, @29=1.08e-06, @30=5.4e-07, @31=-0.00698719, @32=((@10*@2)+(@11*@4)), @33=(0.00646316-@32), @34=((5.689e-05+(((@28*@2)+(@29*@4))-(@30*@4)))-(@31*@33)), @35=(@27*@34), @36=((-1.3e-07+((@5*@4)+-8e-08))+((@8*@12)*@33)), @37=(@36+@36), @38=(0.0037111+(@19*@32)), @39=(-0.00170192-@38), @40=((4.091e-05+(((@35*@17)-(@37*@18))-((@17*@34)-(@18*@36))))-((@14*@22)*@39)), @41=sin(q_3), @42=(((@1*@4)-(@3*@2))*@4), @43=(((0.00017605+(((3.718e-05-@42)-@6)+@7))+(@8*(sq(@12)+sq(@33))))-@13

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [9]:
model.backend.jacobian()

SX(@1=0.673326, @2=2.47e-06, @3=sin(q_0), @4=-0.00025198, @5=cos(q_0), @6=1.19132, @7=-0.110126, @8=cos(q_1), @9=(@3*@8), @10=0.00240029, @11=0.00158266, @12=sin(q_1), @13=(@3*@12), @14=0.839409, @15=0.106092, @16=cos(q_2), @17=sin(q_2), @18=((@9*@16)-(@13*@17)), @19=-0.00541815, @20=0.0347638, @21=((@9*@17)+(@13*@16)), @22=-0.35, @23=(@22*@9), @24=0.564046, @25=0.0436668, @26=cos(q_3), @27=sin(q_3), @28=((@18*@26)-(@21*@27)), @29=0.00364738, @30=-0.00170192, @31=((@18*@27)+(@21*@26)), @32=0.218, @33=0.057, @34=(@23+((@32*@18)+(@33*@21))), @35=0.389385, @36=0.0312153, @37=cos(q_4), @38=sin(q_4), @39=((@28*@37)+(@5*@38)), @40=0.00646316, @41=0.07, @42=(@34+(@41*@28)), @43=0.288758, @44=0.0241569, @45=-0.00017355, @46=((@5*@37)-(@28*@38)), @47=cos(q_5), @48=sin(q_5), @49=-0.00143876, @50=0.0492, @51=(@42+(@50*@39)), @52=(@43*(((@44*@39)+((@45*((@46*@47)+(@31*@48)))+(@49*((@31*@47)-(@46*@48)))))+@51)), @53=((@35*(((@36*@39)+(@40*@31))+@42))+@52), @54=((@24*(((@25*@28)+((@29*@5)+(@30*@31))

In [10]:
model.energy

Energy(kinetic=Function(kinetic_energy:(q[6],v[6])->(kinetic_energy) SXFunction), potential=Function(potential_energy:(q[6])->(potential_energy) SXFunction))

In [11]:
com_position = model.com.position
com_velocity = model.com.velocity
com_jacobian = model.com.jacobian
com_jacobian_dt = model.com.jacobian_dt
potential_energy = model.energy.potential
kinetic_energy = model.energy.kinetic
lagrangian = model.lagrangian

# TODO
# momentum

In [12]:
com_jacobian_dt

Function(com_jacobian_dt:(q[6],v[6])->(com_jacobian_dt[3x6]) SXFunction)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [13]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [14]:
model.inverse_dynamics

Function(inverse_dynamics:(q[6],v[6],dv[6])->(tau[6]) SXFunction)

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [15]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [16]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6])->(dv[6]) SXFunction)

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [17]:
import numpy as np

print(f"Old input dimensions: {model.nu}")
S = np.random.randn(model.nv, model.nv + 2)
model.update_selector(S)
print(f"New input dimensions: {model.nu}")

Old input dimensions: 6
New input dimensions: 8


In [18]:
model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7])

In [19]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[8])->(dv[6]) SXFunction)

In [20]:
model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {model.nu}")
print(f"New selector:\n {model.selector}")

New input dimensions: 3
New selector:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [21]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3])->(dv[6]) SXFunction)

#### **State space Representation and Linearization**

One can easily transform the mechanical system to the state space form by defining the state $\mathbf{x} = [\mathbf{q}, \mathbf{v}]^T$:


$$
\dot{\mathbf{x}}= \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\dot{\mathbf{v}}
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{q})\mathbf{v} \\ 
\text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{S}\mathbf{u})
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{x}_1)\mathbf{x}_2 \\
\text{aba}(\mathbf{x}_1, \mathbf{x}_2, \mathbf{S}\mathbf{u})
\end{bmatrix}
$$



In [22]:
model.state_space

The above equation can be easily linearized to produce following linear approximation:

One can easily find linearization with respect to state:

In [23]:
model.state_space.state_derivative

AttributeError: 'FunctionalStateSpace' object has no attribute 'state_derivative'

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3])->(state_jacobian[12x12,72nz]) SXFunction)

and control:

In [ ]:
model.state_space.input_jacobian

Function(input_jacobian:(q[6],v[6],tau[3])->(input_jacobian[12x3,18nz]) SXFunction)

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06")

One may also retrieve a hash map of all bodies:

In [ ]:
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06").linear_acceleration

FrameQuantity(local=Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world=Function(linear_acceleration_world:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world_aligned=Function(linear_acceleration_world_aligned:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction))

In [ ]:
model.body("link06").linear_acceleration.local

Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

In [ ]:
model.body("link06").jacobian.local

Function(jacobian_local:(i0[6])->(o0[6x6]) SXFunction)

In [ ]:
model.body("link06").jacobian.local
model.body("link06").jacobian_dt.local
model.body("link06").linear_velocity.local
model.body("link06").angular_velocity.local
model.body("link06").linear_acceleration.local
model.body("link06").angular_acceleration.local

Function(angular_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Function(jacobian_world:(i0[6])->(o0[6x6]) SXFunction)

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Function(qforce:(i0[6],i1[6])->(o0[6]) SXFunction)

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3],link06[6])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

Function(state_derivative:(q[6],v[6],tau[3],link06[6])->(state_derivative[12]) SXFunction)

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3],link06[6])->(state_jacobian[12x12,78nz]) SXFunction)

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

Function(nonlin_wrench_cone:(force[6])->(constraint[6]) SXFunction)

In [ ]:
model.body("link06").contact.cone.linear()

DM(
[[-1, 0, -0.5, 0, 0, 0], 
 [1, 0, -0.5, 0, 0, 0], 
 [0, -1, -0.5, 0, 0, 0], 
 [0, 1, -0.5, 0, 0, 0], 
 [0, 0, -0.02, -1, 0, 0], 
 [0, 0, -0.02, 1, 0, 0], 
 [0, 0, -0.05, 0, -1, 0], 
 [0, 0, -0.05, 0, 1, 0], 
 [-0.02, -0.05, -0.035, 0.5, 0.5, -1], 
 [-0.02, 0.05, -0.035, 0.5, -0.5, -1], 
 [0.02, -0.05, -0.035, -0.5, 0.5, -1], 
 [0.02, 0.05, -0.035, -0.5, -0.5, -1], 
 [0.02, 0.05, -0.035, 0.5, 0.5, 1], 
 [0.02, -0.05, -0.035, 0.5, -0.5, 1], 
 [-0.02, 0.05, -0.035, -0.5, 0.5, 1], 
 [-0.02, -0.05, -0.035, -0.5, -0.5, 1]])

In [ ]:
model.add_body({"ee": "link06"})
model.bodies.keys()

dict_keys(['link06', 'ee'])

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.